In [1]:
%reload_ext autoreload
%autoreload 1
import torch 
import sys
sys.path.append('..')
from torch import nn 
from torch.nn import functional as F
from torch import optim
from utils.loader import load,PairSetMNIST,Training_set,Test_set, Training_set_split,Validation_set
from torch.utils.data import Dataset, DataLoader
from models.Le_Net import  LeNet_sharing, train_Le_Net_Weight_Sharing
from models.Inception_Net import Inception_block, conv_block, Google_Net, Auxiliary_loss, train_inception
from utils.evaluate_ws import compute_metrics, validate_model, evaluate_model, grid_search
import torch.cuda as cuda 

In [2]:
train_results, test_losses, test_accuracies = grid_search(LeNet_sharing,train_Le_Net_Weight_Sharing, mini_batch_size=100, optimizer = optim.Adam,criterion = nn.CrossEntropyLoss(), n_epochs=40, eta=1e-3, lambda_l2 = 0,rotate = False,translate=False,swap_channel = False,GPU=True)

prob : 0.1, nb_hidden : 50 (n= 0)
Train Epoch: 0  | Loss 5.514667
Train Epoch: 1  | Loss 5.255795
Train Epoch: 2  | Loss 4.644299
Train Epoch: 3  | Loss 3.744817
Train Epoch: 4  | Loss 3.225718
Train Epoch: 5  | Loss 2.811384
Train Epoch: 6  | Loss 2.475531
Train Epoch: 7  | Loss 2.170223
Train Epoch: 8  | Loss 1.896981
Train Epoch: 9  | Loss 1.542302
Train Epoch: 10  | Loss 1.312851
Train Epoch: 11  | Loss 1.020922
Train Epoch: 12  | Loss 0.858671
Train Epoch: 13  | Loss 0.542441
Train Epoch: 14  | Loss 0.525689
Train Epoch: 15  | Loss 0.675120
Train Epoch: 16  | Loss 0.462371
Train Epoch: 17  | Loss 0.389272
Train Epoch: 18  | Loss 0.217429
Train Epoch: 19  | Loss 0.215670
Train Epoch: 20  | Loss 0.152709
Train Epoch: 21  | Loss 0.134382
Train Epoch: 22  | Loss 0.097012
Train Epoch: 23  | Loss 0.087590
Train Epoch: 24  | Loss 0.071809
Train Epoch: 25  | Loss 0.067720
Train Epoch: 26  | Loss 0.093382
Train Epoch: 27  | Loss 0.069771
Train Epoch: 28  | Loss 0.040970
Train Epoch: 29  | 

In [3]:
train_results.shape

torch.Size([2, 2, 3, 4, 40])

In [25]:
accs = torch.mean(train_results[:,:,:,3,39], dim=2)
losses = torch.mean(train_results[:,:,:,2,39], dim=2)

In [68]:
accs

tensor([[84.1667, 85.3333],
        [83.8333, 82.6667]])

In [74]:
value, idx = accs.max(dim=1)

In [76]:
val, idy = accs.max(dim=0)

In [77]:
idx

tensor([1, 0])

In [78]:
idy

tensor([0, 0])